In [1]:
import json
from pprint import pprint
import collections
from progressbar import ProgressBar
from ripple_api import RippleDataAPIClient

# 1. First hop

In [2]:
data = []
with open('more-txs-from-missing-parsed.json') as f:
    for line in f:
        data.append(json.loads(line))
with open('final_txs_2017.json') as f:
    for line in f:
        data.append(json.loads(line))        

In [3]:
wallets_first_hop = collections.defaultdict(list)
pbar = ProgressBar()
for txn in pbar(data):
    sender = txn["sender"]
    try:
        for link in txn["links"]:
            if link["node1"] == sender:
                wallets_first_hop[sender].append(link["node2"])
    except KeyError:
        continue 

100% |########################################################################|


In [6]:
wallet_two_most_hops = {}
pbar = ProgressBar()
count_unique_first_hop = 0
count_unique_first_hop_if_more_than_one_transaction = 0
for k in pbar(list(wallets_first_hop.keys())):
    counter=collections.Counter(wallets_first_hop[k])
    wallet_two_most_hops[k] = counter.most_common(2)
    if(len(wallet_two_most_hops[k]) == 1):
        count_unique_first_hop += 1
        for a,b in wallet_two_most_hops[k]:
            if(b != 1):
                count_unique_first_hop_if_more_than_one_transaction += 1

100% |########################################################################|


In [14]:
print("There are " + str(count_unique_first_hop/len(wallet_two_most_hops)*100) + "% of wallets that uses always same first hop" +
      " and only " + str(count_unique_first_hop_if_more_than_one_transaction/len(wallet_two_most_hops)*100) + "% if we consider wallets that did more than 1 transaction")

There are 85.15420507567998% of wallets that uses always same first hop and only 40.51439626721292% if we consider wallets that did more than 1 transaction


In [15]:
first_hops = []
pbar = ProgressBar()
for k in pbar(list(wallets_first_hop.keys())):
    for hop in list(set(wallets_first_hop[k])):
        if hop not in first_hops:
            first_hops.append(hop)

100% |########################################################################|


In [16]:
api = RippleDataAPIClient('https://data.ripple.com')

In [17]:
json_gateways = api.get_all_gateways()

In [18]:
gateways_wallet = []
pbar = ProgressBar()
for k in pbar(list(json_gateways.keys())):
    for gateway in json_gateways[k]:
        gateways_wallet.append(gateway["account"])

100% |########################################################################|


In [19]:
inter = list(set(gateways_wallet) & set(first_hops))

In [20]:
print("From the first hops only "+ str(len(inter)/len(first_hops)*100) + "% are know gateways.")

From the first hops only 4.644412191582003% are know gateways.


# 2. No hops

In [46]:
all_senders = set()
sender_with_paths = set()
all_txn = set()
txn_with_paths = set()
pbar = ProgressBar()
for txn in pbar(data):
    sender = txn["sender"]
    txn_hash = txn["hash"]
    all_senders.add(sender)
    all_txn.add(txn_hash)
    if "paths" in txn.keys():
        sender_with_paths.add(sender)  
        txn_with_paths.add(txn_hash)

100% |########################################################################|


In [48]:
sender_with_only_two_hops = list(all_senders - sender_with_paths)
txn_with_only_two_hops = list(all_txn - txn_with_paths)

In [49]:
print("We have "+ str(len(sender_with_only_two_hops)/len(all_senders)*100) + "% of senders that never use paths")

We have 87.98582218291993% of senders that never use paths


In [51]:
print("We have "+ str(len(txn_with_only_two_hops)/len(all_txn)*100) + "% of transactions that never use paths")

We have 78.80077420297265% of transactions that never use paths
